In [30]:
""" 
Assessing greenspace accessability within Northern Ireland through an interactive map which calculates the nearest proximity of a settlement to a greenspace trail. 
"""
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely.strtree import STRtree
from shapely.ops import nearest_points

In [31]:
"""
Load settlement, trail, and greenspace shapefiles.
"""
settlements = gpd.read_file("data/Settlements/settlements-2015-above-500-threshold.shp")
trails = gpd.read_file("data/Trails/greenspaceoffroadtrails.shp")
greenspace = gpd.read_file("data/Greenspace/greenspace.shp")


In [32]:
"""
Check the EPSG for the settlements data file.
"""
settlements.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [33]:
"""
Check the EPSG for the trails data file.
"""
trails.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [34]:
"""
Check the EPSG for the greenspace data file.
"""
greenspace.crs

<Projected CRS: EPSG:29902>
Name: TM65 / Irish Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Ireland - onshore.
- bounds: (-10.56, 51.39, -5.93, 55.43)
Coordinate Operation:
- name: Irish Grid
- method: Transverse Mercator
Datum: TM65
- Ellipsoid: Airy Modified 1849
- Prime Meridian: Greenwich

In [35]:
"""
Reproject all spatial layers to EPSG:3857 for compatibility with calculations. 
"""
settlements = settlements.to_crs(epsg=3857)
trails = trails.to_crs(epsg=3857)
greenspace = greenspace.to_crs(epsg=3857)

In [36]:
"""
Re-check the EPSG for the settlements data file.
"""
settlements.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [37]:
"""
Re-check the EPSG for the trails data file.
"""
trails.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [38]:
"""
Re-check the EPSG for the greenspace data file.
"""
greenspace.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [39]:
"""
Display the first 5 rows of the greenspace shapefile.
"""
greenspace.head(5)

,SourceID,GUID,Name,Source,Category,Type,PaidAccess,Area_Ha,Verified,ShowOnMap,ORNI_ID,DataAdded,SiteCreate,geometry
0,None,{68D2DF20-1512-4218-A759-1000732E93B0},Conservation Volunteers NI,LPS and Outscape,Amenity Greenspace,Community Garden,No,0.668234,Approximated,Yes,51,2022-09-21,Pre 2023,"POLYGON Z ((-658305.131 7272616.366 0, -658467..."
1,None,{7B20F220-682E-4566-A4FE-2513CC65ED6E},Lough Shore Park,Antrim and Newtownabbey Borough Council,Parks and Gardens,Public Park,No,6.598284,Approximated,Yes,61,2022-09-21,Pre 2023,"POLYGON Z ((-694569.114 7306677.129 0, -694647..."
2,None,{8A4B8A31-EBA9-4E7A-B8D5-CEA0724D848D},Ardmore Recreation Centre,"Armagh City, Banbridge and Craigavon Borough C...",Amenity Greenspace,Playing Fields,No,2.969549,Approximated,Yes,67,2022-09-21,Pre 2023,"POLYGON Z ((-737723.904 7234648.427 0, -737711..."
3,None,{36F2A040-649C-4EB5-9174-7B25C083F489},Clare Glen - Phase 3 - Link Footpath/River Wal...,"Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,3.396109,Approximated,Yes,69,2022-09-21,Pre 2023,"POLYGON Z ((-713809.418 7236506.822 0, -713812..."
4,None,{2F586E46-B9EE-4D13-83B1-9B254FA3A698},"Folly Glen, Armagh","Armagh City, Banbridge and Craigavon Borough C...",Woodland,Woodland,No,7.849400,Approximated,Yes,70,2022-09-21,Pre 2023,"MULTIPOLYGON Z (((-738837.244 7234074.082 0, -..."


In [40]:
"""
Display the first 5 rows of the settlements shapefile.
"""
settlements.head(5)

,Code,Name,Band,DT_20MIN,DT_30MIN,UR_ex,OccHH_ex,UR_ap,OccHH_ap,geometry
0,N11000199,DUNNAMORE,H,Y,Y,119.0,35.0,119.0,35.0,"MULTIPOLYGON (((-771836.094 7298554.224, -7717..."
1,N11000015,BALLYBARNES,H,Y,Y,242.0,101.0,243.0,102.0,"POLYGON ((-639797.918 7285752.046, -639794.956..."
2,N11000121,LOUGHGUILE,H,N,Y,396.0,128.0,396.0,128.0,"POLYGON ((-702052.069 7374194.437, -702048.944..."
3,N11000265,BRYANSFORD,H,N,Y,306.0,114.0,309.0,115.0,"POLYGON ((-661378.035 7213541.816, -661374.07 ..."
4,N11000636,SION MILLS,G,Y,Y,1903.0,769.0,1907.0,770.0,"POLYGON ((-832102.671 7319464.147, -832105.467..."


In [41]:
"""
Display the first 5 rows of the trails shapefile.
"""
trails.head(5)

,ID,Name,SourceID,Source,GUID,Category,Type,Descriptio,On_Road,Length,Verified,ShowOnMap,County,UpdateDate,Trail_Usea,Infrastruc,geometry
0,0,Annalong Coastal Path,166,Outscape,{7a03aac4-145f-436f-9320-45b533fc5930},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.055094,None,Yes,Down,None,Walking,False,"LINESTRING Z (-656477.346 7190872.985 0, -6564..."
1,0,Antrim Hills Way,104,Outscape,{c89d5023-1fe8-4def-b9d6-9efdc833540a},Single-Use Trail,Walking Trail,Long Walks (over 20 miles),False,23.081263,None,Yes,Antrim,None,Walking,False,"LINESTRING Z (-662985.466 7355853.417 0, -6629..."
2,0,Ardress House - Ladies Mile,149,Outscape,{0a2fe306-5136-48e1-9a09-067c5dffe2d8},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.949078,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-733785.916 7254732.363 0, -7337..."
3,0,Argory Blackwater River Walk,724,Outscape,{4b801c6c-fab1-4b9e-a475-e2c79be826bc},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,1.546865,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-740877.981 7258706.199 0, -7408..."
4,0,Argory Lime Tree Walk,319,Outscape,{3c967a01-0486-4f4c-9924-2d9a888e087a},Single-Use Trail,Walking Trail,Short Walks (up to 5 miles),False,0.955559,None,Yes,Armagh,None,Walking,False,"LINESTRING Z (-740877.981 7258706.199 0, -7408..."


In [42]:
"""
Filter out invalid geometries to avoid errors during distance calculations,
spatial indexing, and map rendering.
"""
settlements = settlements[settlements.is_valid]
trails = trails[trails.is_valid]
greenspace = greenspace[greenspace.is_valid]

In [43]:
"""
Calculate area (in hectares) for each greenspace polygon and store its centroid.
"""
if 'Area_Ha' not in greenspace.columns:
    greenspace['Area_Ha'] = greenspace.geometry.area / 10_000  # Convert m² to ha

greenspace['centroid'] = greenspace.geometry.centroid

In [44]:
"""
Use a spatial index (STRtree) to efficiently find the nearest trail segment
to each settlement. Record both the connecting line and the distance.
"""
trail_geoms = trails.geometry.tolist() # Converts trail geometries to a list
trail_tree = STRtree(trail_geoms) # Builds a spatial index

connection_lines = []  # List to hold LineStrings from settlements to trails
distances = []         # List to hold Euclidean distances (m)

for idx, row in settlements.iterrows():
    point = row.geometry.centroid
    nearest_idx = trail_tree.nearest(point)
    nearest_geom = trail_geoms[nearest_idx]
    nearest_point = nearest_points(point, nearest_geom)[1]

    line = LineString([point, nearest_point]) # Creates connecting lines
    distance = point.distance(nearest_point) # Measures the distance from the nearest trails to surrounding settlements

    connection_lines.append(line)
    distances.append(distance)

# Create GeoDataFrame of connection lines and distances
connections = gpd.GeoDataFrame(
    {'distance_m': distances},
    geometry=connection_lines,
    crs=settlements.crs # Use the same CRS as settlements
)

In [45]:
"""
Check the EPSG of the connections.gpd.
"""
connections.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [46]:
"""
Reproject all spatial layers to EPSG:4326 for compatibility with Folium.
"""
settlements = settlements.to_crs(epsg=4326)
trails = trails.to_crs(epsg=4326)
greenspace = greenspace.to_crs(epsg=4326)
connections = connections.to_crs(epsg=4326)

In [47]:
"""
Re-check the EPSG of the settlements data file.
"""
settlements.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [48]:
"""
Re-check the EPSG of the trails data file.
"""
trails.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
"""
Re-check the EPSG of the greenspace data file.
"""
greenspace.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [50]:
"""
Re-check the EPSG of the connections data file.
"""
connections.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [51]:
"""
Center the map on the geographic centroid of all settlements and initialize a
Folium web map.
"""
center = settlements.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=8)


In [52]:
"""
Overlay greenspace polygons on the map with popups showing name, category and area.
"""
for _, row in greenspace.iterrows():
    popup = folium.Popup(
        f"<strong>Name:</strong> {row.get('Name', 'N/A')}<br>"
        f"<strong>Category:</strong> {row.get('Category','N/A')}<br>"
        f"<strong>Area:</strong> {round(row['Area_Ha'], 2)} ha",
        max_width=250
    )
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'darkgreen',
            'fillOpacity': 0.4,
            'weight': 0.5
        },
        popup=popup
    ).add_to(m)


In [53]:
"""
Add trail network lines to the map that are clickable and show the trail names.
"""
folium.GeoJson(
    trails,
    name='Trails',
    style_function=lambda x: {
        'color': 'blue',
        'weight': 2
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['Name'],           
        aliases=['Trail Name:'],
        localize=True,
        sticky=True
    )
).add_to(m)

In [54]:
"""
Represent each settlement with a black circle marker, using the centroid for placement.
"""
for _, row in settlements.iterrows():
    centroid = row.geometry.centroid
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=4,
        color='black',
        fill=True,
        fill_opacity=0.7,
        tooltip=row.get("Name", "N/A")
    ).add_to(m)

In [55]:
"""
Draw red lines from each settlement to the nearest trail, with tooltip showing distance.
"""
for _, row in connections.iterrows():
    folium.GeoJson(
        row.geometry,
        name='Nearest Trail Line',
        style_function=lambda x: {
            'color': 'red',
            'weight': 1.5
        },
        tooltip=f"Distance: {int(row['distance_m'])} m"
    ).add_to(m)


In [56]:
"""
Add interactive layer control and export the complete map as an HTML file.
"""
folium.LayerControl().add_to(m)
m.save("nearest_trail_access_map.html")

print('nearest_trail_access_map.html')

nearest_trail_access_map.html
